# 🌲 Forest Pattern Recognition Tutorial

**Random Forest Classifier for Deforestation Risk Assessment**

This notebook walks through building a machine learning system to classify forest areas by risk level based on:
- NDVI (vegetation health)
- Texture features (canopy structure)
- Spatial patterns (fragmentation)

## Outline:
1. Setup and Data Generation
2. Feature Engineering
3. Model Training
4. Evaluation
5. Risk Map Generation
6. Real-World Application

## 1. Setup 📦

In [ ]:
# Install if needed
# !pip install scikit-learn numpy pandas matplotlib seaborn scipy scikit-image

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ Libraries loaded successfully")

## 2. Understanding NDVI 🌿

**NDVI (Normalized Difference Vegetation Index)** measures vegetation health:

$$NDVI = \frac{NIR - Red}{NIR + Red}$$

- **NIR**: Near-Infrared reflectance
- **Red**: Red band reflectance

### Interpretation:
- **0.6 to 1.0**: Dense, healthy vegetation 🌲
- **0.2 to 0.6**: Sparse vegetation 🌾
- **-1.0 to 0.2**: No vegetation (water, bare soil, urban) 🏜️

In [ ]:
def calculate_ndvi(nir, red):
    """Calculate NDVI from NIR and Red bands"""
    return (nir - red) / (nir + red + 1e-10)

# Example calculation
nir_healthy = 0.5
red_healthy = 0.1
ndvi_healthy = calculate_ndvi(nir_healthy, red_healthy)

nir_degraded = 0.2
red_degraded = 0.3
ndvi_degraded = calculate_ndvi(nir_degraded, red_degraded)

print(f"Healthy Forest NDVI: {ndvi_healthy:.3f}")
print(f"Degraded Area NDVI: {ndvi_degraded:.3f}")

## 3. Generate Synthetic Data 🎲

We'll create synthetic forest patches with different risk levels.

In [ ]:
def generate_forest_patch(risk_level, size=32):
    """
    Generate a synthetic NDVI patch
    
    Args:
        risk_level: 'Low', 'Medium', or 'High'
        size: patch dimension
    """
    if risk_level == 'Low':
        # Healthy forest: high NDVI, low variability
        base = np.random.uniform(0.65, 0.85)
        noise = np.random.normal(0, 0.05, (size, size))
        
    elif risk_level == 'Medium':
        # Some disturbance: medium NDVI, moderate variability
        base = np.random.uniform(0.35, 0.55)
        noise = np.random.normal(0, 0.1, (size, size))
        
        # Add degradation patches
        patch = np.clip(base + noise, 0, 1)
        for _ in range(np.random.randint(1, 3)):
            x, y = np.random.randint(0, size-10, 2)
            w, h = np.random.randint(5, 12, 2)
            patch[y:y+h, x:x+w] *= np.random.uniform(0.4, 0.7)
        return patch
        
    else:  # High risk
        # Severe degradation: low NDVI, high variability
        base = np.random.uniform(0.1, 0.35)
        noise = np.random.normal(0, 0.15, (size, size))
        
        # Add large degraded areas
        patch = np.clip(base + noise, 0, 1)
        for _ in range(np.random.randint(2, 5)):
            x, y = np.random.randint(0, size-15, 2)
            w, h = np.random.randint(8, 20, 2)
            patch[y:y+h, x:x+w] *= np.random.uniform(0.1, 0.4)
        return patch
    
    return np.clip(base + noise, 0, 1)

# Generate samples
n_samples_per_class = 100
patch_size = 32

images = []
labels = []

for risk in ['Low', 'Medium', 'High']:
    for _ in range(n_samples_per_class):
        patch = generate_forest_patch(risk, patch_size)
        images.append(patch)
        labels.append(risk)

images = np.array(images)
labels = np.array(labels)

print(f"✓ Generated {len(images)} forest patches")
print(f"  Shape: {images.shape}")
print(f"  Classes: {np.unique(labels)}")

In [ ]:
# Visualize samples
fig, axes = plt.subplots(3, 4, figsize=(14, 10))

for i, risk in enumerate(['Low', 'Medium', 'High']):
    risk_indices = np.where(labels == risk)[0]
    
    for j in range(4):
        idx = risk_indices[j]
        patch = images[idx]
        
        im = axes[i, j].imshow(patch, cmap='RdYlGn', vmin=0, vmax=1)
        axes[i, j].set_title(f'{risk} Risk\nMean NDVI: {patch.mean():.2f}')
        axes[i, j].axis('off')
        
        if j == 3:
            plt.colorbar(im, ax=axes[i, j], fraction=0.046)

plt.tight_layout()
plt.show()

## 4. Feature Engineering 🔧

Extract meaningful features from NDVI patches.

In [ ]:
def extract_features(ndvi_patch):
    """
    Extract features from an NDVI patch
    """
    features = {}
    
    # Basic statistics
    features['mean'] = np.mean(ndvi_patch)
    features['std'] = np.std(ndvi_patch)
    features['min'] = np.min(ndvi_patch)
    features['max'] = np.max(ndvi_patch)
    features['median'] = np.median(ndvi_patch)
    features['range'] = features['max'] - features['min']
    
    # Percentiles
    features['p25'] = np.percentile(ndvi_patch, 25)
    features['p75'] = np.percentile(ndvi_patch, 75)
    features['iqr'] = features['p75'] - features['p25']
    
    # Vegetation categories
    dense = (ndvi_patch >= 0.6).sum() / ndvi_patch.size
    sparse = ((ndvi_patch >= 0.3) & (ndvi_patch < 0.6)).sum() / ndvi_patch.size
    bare = (ndvi_patch < 0.3).sum() / ndvi_patch.size
    
    features['dense_veg_ratio'] = dense
    features['sparse_veg_ratio'] = sparse
    features['bare_ratio'] = bare
    
    # Texture (simple approximation)
    features['texture_h'] = np.std(np.diff(ndvi_patch, axis=0))
    features['texture_v'] = np.std(np.diff(ndvi_patch, axis=1))
    
    # Coefficient of variation
    features['cv'] = features['std'] / (features['mean'] + 1e-7)
    
    return features

# Extract features for all samples
print("Extracting features...")
feature_list = []
for patch in images:
    feats = extract_features(patch)
    feature_list.append(feats)

# Convert to DataFrame
features_df = pd.DataFrame(feature_list)
features_df['label'] = labels

print(f"✓ Extracted {len(features_df.columns)-1} features")
print("\nFeature names:")
print(list(features_df.columns[:-1]))

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

important_features = ['mean', 'std', 'dense_veg_ratio', 'texture_h', 'cv', 'range']

for i, feat in enumerate(important_features):
    for risk in ['Low', 'Medium', 'High']:
        data = features_df[features_df['label'] == risk][feat]
        axes[i].hist(data, alpha=0.5, label=risk, bins=20)
    
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('Frequency')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice how features differ between risk classes!")

## 5. Model Training 🚀

In [ ]:
# Prepare data
X = features_df.drop('label', axis=1).values
y = features_df['label'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Data prepared and scaled")

In [ ]:
# Train Random Forest
print("Training Random Forest Classifier...\n")

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

model.fit(X_train_scaled, y_train)

print("\n✓ Training complete!")

## 6. Model Evaluation 📊

In [ ]:
# Predictions
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n{'='*60}")
print(f"Overall Accuracy: {accuracy:.4f}")
print(f"{'='*60}\n")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=['Low', 'Medium', 'High'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low', 'Medium', 'High'],
            yticklabels=['Low', 'Medium', 'High'])
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Feature Importance
feature_names = features_df.columns[:-1]
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Importance')
plt.title('Feature Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
for i in range(5):
    idx = indices[i]
    print(f"{i+1}. {feature_names[idx]}: {importances[idx]:.4f}")

## 7. Cross-Validation 🔄

In [ ]:
# 5-Fold Cross-Validation
print("Running 5-fold cross-validation...\n")

cv_scores = cross_val_score(
    model, 
    scaler.transform(X),
    y,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

print("Cross-Validation Scores:")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")

print(f"\nMean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 8. Risk Map Generation 🗺️

Apply the model to a large area by dividing it into grid cells.

In [ ]:
def create_risk_map(model, scaler, ndvi_map, grid_size=32):
    """
    Generate risk classification map
    """
    h, w = ndvi_map.shape
    n_rows = h // grid_size
    n_cols = w // grid_size
    
    risk_map = np.empty((n_rows, n_cols), dtype=object)
    prob_map = np.zeros((n_rows, n_cols, 3))  # Low, Medium, High
    
    print(f"Processing {n_rows}x{n_cols} = {n_rows*n_cols} grid cells...")
    
    for i in range(n_rows):
        for j in range(n_cols):
            # Extract cell
            cell = ndvi_map[
                i*grid_size:(i+1)*grid_size,
                j*grid_size:(j+1)*grid_size
            ]
            
            # Extract features
            feats = extract_features(cell)
            X_cell = np.array([list(feats.values())])
            X_cell_scaled = scaler.transform(X_cell)
            
            # Predict
            risk_map[i, j] = model.predict(X_cell_scaled)[0]
            prob_map[i, j] = model.predict_proba(X_cell_scaled)[0]
    
    print("✓ Risk map generated")
    return risk_map, prob_map

# Generate synthetic large area
large_size = 512
print("\nGenerating synthetic NDVI map...")

ndvi_large = np.zeros((large_size, large_size))

# Healthy forest (top-left)
ndvi_large[:256, :256] = np.clip(np.random.normal(0.72, 0.08, (256, 256)), 0, 1)

# Medium risk (top-right)
ndvi_large[:256, 256:] = np.clip(np.random.normal(0.45, 0.12, (256, 256)), 0, 1)

# High risk (bottom)
ndvi_large[256:, :] = np.clip(np.random.normal(0.25, 0.15, (256, 512)), 0, 1)

print(f"✓ NDVI map created: {ndvi_large.shape}")

# Generate risk map
risk_map, prob_map = create_risk_map(model, scaler, ndvi_large, grid_size=32)

In [ ]:
# Visualize Risk Map
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Original NDVI
im0 = axes[0, 0].imshow(ndvi_large, cmap='RdYlGn', vmin=0, vmax=1)
axes[0, 0].set_title('NDVI Map', fontsize=12, fontweight='bold')
plt.colorbar(im0, ax=axes[0, 0])

# Risk Classification
risk_numeric = np.vectorize({'Low': 0, 'Medium': 1, 'High': 2}.get)(risk_map)
im1 = axes[0, 1].imshow(risk_numeric, cmap='RdYlGn_r', vmin=0, vmax=2)
axes[0, 1].set_title('Risk Classification', fontsize=12, fontweight='bold')
cbar = plt.colorbar(im1, ax=axes[0, 1], ticks=[0, 1, 2])
cbar.set_ticklabels(['Low', 'Medium', 'High'])

# High Risk Probability
im2 = axes[1, 0].imshow(prob_map[:, :, 2], cmap='Reds', vmin=0, vmax=1)
axes[1, 0].set_title('High Risk Probability', fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=axes[1, 0])

# Statistics
axes[1, 1].axis('off')
total = risk_map.size
low = (risk_map == 'Low').sum()
medium = (risk_map == 'Medium').sum()
high = (risk_map == 'High').sum()

stats_text = f"""
RISK ASSESSMENT SUMMARY
{'='*40}

Total Cells: {total}

Low Risk:    {low:3d} ({low/total*100:5.1f}%)
Medium Risk: {medium:3d} ({medium/total*100:5.1f}%)
High Risk:   {high:3d} ({high/total*100:5.1f}%)

{'='*40}

⚠️  Critical Areas: {high} cells
   require immediate attention

Mean NDVI: {ndvi_large.mean():.3f}
"""

axes[1, 1].text(0.1, 0.5, stats_text, 
                fontsize=11, 
                family='monospace',
                verticalalignment='center',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 9. Save Model 💾

In [ ]:
import joblib

# Save model and scaler
joblib.dump({
    'model': model,
    'scaler': scaler,
    'feature_names': list(feature_names)
}, 'forest_risk_classifier.pkl')

print("✓ Model saved: forest_risk_classifier.pkl")
print("\nTo load later:")
print(">>> data = joblib.load('forest_risk_classifier.pkl')")
print(">>> model = data['model']")
print(">>> scaler = data['scaler']")

## 10. Real-World Usage Example 🌍

In [ ]:
def predict_single_area(model, scaler, ndvi_patch):
    """
    Predict risk for a single area
    
    Args:
        model: trained classifier
        scaler: fitted StandardScaler
        ndvi_patch: 2D NDVI array
    
    Returns:
        prediction, probabilities
    """
    # Extract features
    feats = extract_features(ndvi_patch)
    X = np.array([list(feats.values())])
    X_scaled = scaler.transform(X)
    
    # Predict
    prediction = model.predict(X_scaled)[0]
    probabilities = model.predict_proba(X_scaled)[0]
    
    # Format results
    class_names = model.classes_
    prob_dict = {cls: prob for cls, prob in zip(class_names, probabilities)}
    
    return prediction, prob_dict

# Example: Test on new data
print("Testing on new forest patches:\n")

# Test 1: Healthy forest
test_healthy = generate_forest_patch('Low', 32)
pred, probs = predict_single_area(model, scaler, test_healthy)
print(f"Test 1 - Healthy Forest:")
print(f"  Mean NDVI: {test_healthy.mean():.3f}")
print(f"  Prediction: {pred}")
print(f"  Confidence: {max(probs.values()):.1%}")
print()

# Test 2: Degraded area
test_degraded = generate_forest_patch('High', 32)
pred, probs = predict_single_area(model, scaler, test_degraded)
print(f"Test 2 - Degraded Area:")
print(f"  Mean NDVI: {test_degraded.mean():.3f}")
print(f"  Prediction: {pred}")
print(f"  Confidence: {max(probs.values()):.1%}")

## 🎉 Summary

In this tutorial, you learned:

1. ✅ What NDVI is and how to interpret it
2. ✅ How to engineer features from satellite imagery
3. ✅ How to train a Random Forest classifier
4. ✅ How to evaluate model performance
5. ✅ How to generate risk classification maps
6. ✅ How to save and deploy the model

### Next Steps:

- **Use Real Data**: Load actual satellite imagery (Sentinel-2, Landsat)
- **Add More Features**: Include elevation, slope, distance to roads
- **Time Series**: Track changes over multiple dates
- **Advanced Models**: Try XGBoost, LightGBM, or deep learning
- **Deploy**: Create a web service or GIS plugin

### Resources:

- [Scikit-learn Documentation](https://scikit-learn.org/)
- [Sentinel-2 Data](https://scihub.copernicus.eu/)
- [Google Earth Engine](https://earthengine.google.com/)

**Happy forest monitoring! 🌲📊**